Task: Recommend 10 unseen songs for every users
Goal: maximize nDCG
Data: user info
      item info
      user interactions with items(test + train)
      item embeddings
Submit: report.txt + codes.zip + recommendations.tsv
TODO: create train-test-val data splits
      setup nDCG evaluation
      produce & evaluate random recommendations
      produce and evaluate POP recommendations

      


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cosine_similarity
from tqdm import tqdm
from typing import Callable, List
from sklearn.metrics import ndcg_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import datetime
from sklearn.preprocessing import LabelEncoder
import random
from torch.utils.data import Dataset, DataLoader


In [2]:
def inter_matr_implicit(users: int,
                       items: int,
                       interactions: pd.DataFrame,
                       threshold=1) -> np.ndarray:
    """
    Create an implicit interaction matrix from user-item interactions.
    
    Parameters:
        users: DataFrame containing user information
        items: DataFrame containing item information
        interactions: DataFrame containing user-item interaction data
        threshold: Minimum value for a valid interaction (default: 1)
        
    Returns:
        2D numpy array where rows represent users and columns represent items
    """
    interactions = interactions.copy()

    n_users = len(users.index)
    n_items = len(items.index)
    res = np.zeros([n_users, n_items], dtype=np.int8)

    row = interactions['user_id'].to_numpy()
    col = interactions["item_id"].to_numpy()

    data = interactions['count'].to_numpy()
    data[data < threshold] = 0
    data[data >= threshold] = 1

    res[row, col] = data

    return res

def get_ndcg_score_sk(df_predictions, test_interaction_matrix: np.ndarray, topK=10) -> float:
    """
    Calculate the NDCG score for recommendation predictions.
    
    Parameters:
        df_predictions: DataFrame containing recommendation predictions
        test_interaction_matrix: Ground truth interaction matrix
        topK: Number of top recommendations to evaluate (default: 10)
        
    Returns:
        Average NDCG score across all users
    """
    ndcg_avg = 0
    
    for _, row in df_predictions.iterrows():
        g_truth = test_interaction_matrix[row["user_id"]]

        predicted_scores = np.zeros(len(g_truth),dtype=np.int8)

        predictions = list(map(int, row["recs"].split(",")))[:topK]

        for j, rec in enumerate(predictions):
            predicted_scores[rec] = topK-j

        ndcg_avg += ndcg_score(g_truth.reshape(1, -1), predicted_scores.reshape(1, -1), k=topK)

    return ndcg_avg/len(df_predictions)


In [3]:
def read(dataset, file):
    return pd.read_csv(dataset + '/' + dataset + '.' + file, sep='\t')

# Load User Data
users = read('lfm-challenge', 'user')
print("Users Data Head:")
#print(users.head())

# Load Item Data
items = read('lfm-challenge', 'item')
print("\nItem Data Head:")
#print(items.head())

# Load Training Interactions
train_inters = read('lfm-challenge', 'inter_train')
print("\nTraining Interactions Head:")
#print(train_inters.head())

# Load Testing Interactions
test_inters = read('lfm-challenge', 'inter_test')
print("\nTesting Interactions Head:")
#print(test_inters.head())

# Load Item Embeddings
item_embedding = read('lfm-challenge', 'musicnn')
print("\nEmbeddings Head:")
print(item_embedding.head())

# Load User Embeddings
user_embedding = read('lfm-challenge', 'usernn')
print("\nEmbeddings Head:")
print(user_embedding.head())

train_interaction_matrix = inter_matr_implicit(users, items, train_inters)
test_interaction_matrix = inter_matr_implicit(users, items, test_inters)

Users Data Head:

Item Data Head:

Training Interactions Head:

Testing Interactions Head:

Embeddings Head:
   item_id         0         1         2         3         4         5  \
0        0  0.221942  0.006455  0.027300  0.091775  0.013135  0.137436   
1        1  0.166340  0.000332  0.018895  0.140315  0.002309  0.111743   
2        2  0.247896  0.003749  0.034527  0.036859  0.008251  0.115214   
3        3  0.229554  0.000968  0.028905  0.027514  0.002186  0.100847   
4        4  0.009760  0.000590  0.008925  0.721381  0.000711  0.073143   

          6         7         8  ...        40        41        42        43  \
0  0.082835  0.275749  0.126342  ...  0.058063  0.014128  0.000574  0.001193   
1  0.102853  0.483104  0.135297  ...  0.191162  0.014372  0.000179  0.000249   
2  0.030934  0.609462  0.058102  ...  0.009187  0.005204  0.000456  0.000602   
3  0.029319  0.564656  0.080171  ...  0.008916  0.004114  0.000110  0.000287   
4  0.454569  0.118651  0.368946  ...  0.008477

In [4]:
# class ImplicitDataset(Dataset):
#     def __init__(self, positive_pairs, n_items, neg_ratio=1):
#         self.positive_pairs = positive_pairs
#         self.n_items = n_items
#         self.neg_ratio = neg_ratio
#         # build a fast lookup set of positives
#         self.pos_set = set(positive_pairs)

#     def __len__(self):
#         return len(self.positive_pairs)

#     def __getitem__(self, idx):
#         u, i_pos = self.positive_pairs[idx]
#         # sample negatives
#         negs = []
#         while len(negs) < self.neg_ratio:
#             i_neg = random.randrange(self.n_items)
#             if (u, i_neg) not in self.pos_set:
#                 negs.append(i_neg)
#         return u, i_pos, negs
    
class ImplicitDataset(Dataset):
    def __init__(self, inter_matr: np.ndarray, n_items:int):
        self.positive_pairs = []
        self.negative_pairs = []
        for u in range(len(inter_matr)):
            self.positive_pairs.extend([(u, i) for i in np.where(inter_matr[u] > 0)])
            self.negative_pairs.extend([(u, i) for i in np.where(inter_matr[u] == 0)])
        self.n_items = n_items
        # build a fast lookup set of positives
        self.pos_set = set(self.positive_pairs)

    def __len__(self):
        return len(self.positive_pairs)

    def __getitem__(self, idx):
        u, i_pos = self.positive_pairs[idx]
        return u, i_pos, self.negative_pairs[idx]
    

class NeuMF(nn.Module):
    """
    Neural Collaborative Filtering (NeuMF) model combining GMF and MLP components.
    Args:
        num_users (int): Total number of users.
        num_items (int): Total number of items.
        gmf_dim (int): Embedding size for the GMF path.
        mlp_emb_dim (int): Embedding size for the MLP path.
        mlp_hidden_layers (list of int): Sizes of the hidden layers in the MLP.
    """
    def __init__(
        self,
        num_users: int,
        num_items: int,
        gmf_dim: int = 32,
        mlp_emb_dim: int = 32,
        mlp_hidden_layers: list = [64, 32, 16, 8]
    ):
        super().__init__()
        # GMF embeddings
        self.gmf_user_emb = nn.Embedding(num_users, gmf_dim)
        self.gmf_item_emb = nn.Embedding(num_items, gmf_dim)
        # MLP embeddings
        self.mlp_user_emb = nn.Embedding(num_users, mlp_emb_dim)
        self.mlp_item_emb = nn.Embedding(num_items, mlp_emb_dim)

        # Build MLP layers
        mlp_input_size = mlp_emb_dim * 2
        mlp_layers = []
        for layer_size in mlp_hidden_layers:
            mlp_layers.append(nn.Linear(mlp_input_size, layer_size))
            mlp_layers.append(nn.ReLU())
            mlp_input_size = layer_size
        self.mlp = nn.Sequential(*mlp_layers)

        # Final prediction layer
        predict_input_size = gmf_dim + mlp_hidden_layers[-1]
        self.predict = nn.Linear(predict_input_size, 1)

    def forward(self, user_ids: torch.LongTensor, item_ids: torch.LongTensor) -> torch.Tensor:
        """
        Forward pass for NeuMF.
        Args:
            user_ids (LongTensor): Tensor of user indices, shape (batch_size,).
            item_ids (LongTensor): Tensor of item indices, shape (batch_size,).
        Returns:
            scores (Tensor): Predicted interaction scores (logits), shape (batch_size,).
        """
        # GMF path
        gmf_u = self.gmf_user_emb(user_ids)  # (B, gmf_dim)
        gmf_i = self.gmf_item_emb(item_ids)  # (B, gmf_dim)
        gmf_out = gmf_u * gmf_i              # (B, gmf_dim)

        # MLP path
        mlp_u = self.mlp_user_emb(user_ids)  # (B, mlp_emb_dim)
        mlp_i = self.mlp_item_emb(item_ids)  # (B, mlp_emb_dim)
        mlp_in = torch.cat([mlp_u, mlp_i], dim=1)  # (B, mlp_emb_dim*2)
        mlp_out = self.mlp(mlp_in)                 # (B, last_hidden)

        # Concatenate GMF and MLP parts
        concat = torch.cat([gmf_out, mlp_out], dim=1)  # (B, gmf_dim + last_hidden)
        logits = self.predict(concat).squeeze(1)       # (B,)
        return logits
def train_neumf(model: NeuMF, train_loader: DataLoader, optimizer: torch.optim.Optimizer, device: torch.device, epochs: int = 10):
    """
    Train the NeuMF model.
    
    Args:
        model (NeuMF): The NeuMF model to train.
        train_loader (DataLoader): DataLoader for training data.
        optimizer (torch.optim.Optimizer): Optimizer for training.
        device (torch.device): Device to run the training on (CPU or GPU).
        epochs (int): Number of training epochs.
    """
    model.train()
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        total_loss = 0.0
        for user_ids, pos_item_ids, neg_item_ids in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            user_ids = user_ids.to(device)
            pos_item_ids = pos_item_ids.to(device)
            neg_item_ids = torch.tensor(neg_item_ids, dtype=torch.long).to(device)

            # Positive scores
            pos_scores = model(user_ids, pos_item_ids)
            # Negative scores
            neg_scores = model(user_ids, neg_item_ids)

            # Create labels
            labels = torch.cat([torch.ones_like(pos_scores), torch.zeros_like(neg_scores)], dim=0).to(device)

            # Concatenate scores
            scores = torch.cat([pos_scores, neg_scores], dim=0)

            # Compute loss
            loss = criterion(scores, labels.float())
            total_loss += loss.item()

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")


In [15]:

def get_negative_samples(user_id, n_items, pos_items_set, num_neg=1):
    """
    Sample num_neg negative items for a user (items not in pos_items_set).
    """
    negs = []
    while len(negs) < num_neg:
        neg_item = random.randint(0, n_items - 1)
        if neg_item not in pos_items_set and neg_item not in negs:
            negs.append(neg_item)
    return negs

class ImplicitDataset(Dataset):
    def __init__(self, inter_matr: np.ndarray, n_items: int, neg_ratio: int = 4):
        self.positive_pairs = []
        self.user_pos_items = {}
        for u in range(len(inter_matr)):
            pos_items = set(np.where(inter_matr[u] > 0)[0])
            self.user_pos_items[u] = pos_items
            self.positive_pairs.extend([(u, i) for i in pos_items])
        self.n_items = n_items
        self.neg_ratio = neg_ratio


    def __len__(self):
        return len(self.positive_pairs)

    def __getitem__(self, idx):
        u, i_pos = self.positive_pairs[idx]
        pos_items_set = self.user_pos_items[u]
        neg_items = get_negative_samples(u, self.n_items, pos_items_set, self.neg_ratio)
        return u, i_pos, neg_items
    
def collate_fn(batch):
    users, pos_items, neg_lists = zip(*batch)
    users = torch.LongTensor(users)
    pos_items = torch.LongTensor(pos_items)
    neg_items = torch.LongTensor([n for neg in neg_lists for n in neg])
    labels = torch.cat([torch.ones(len(users)), torch.zeros(len(neg_items))])
    items = torch.cat([pos_items, neg_items])
    users = users.repeat(1 + len(neg_lists[0]))
    return users, items, labels

np.random.seed(42)

#numbers of users
n_users = len(users)
n_items = len(items)
n_countries = len(users['country'].unique())
n_genders = len(users['gender'].unique())

#pos_inter = list(zip(train_inters['user_id'].to_list(), train_inters['item_id'].to_list())) 
# print(pos_inter)
# print(np.array(pos_inter))
#neg_inter = np.array([[u, i] for u in range(n_users) for i in range(n_items) if train_interaction_matrix[u][i] == 0 ])

print("pos, neg inter completed")
# Use the interaction matrix as input to ImplicitDataset
dataset_train = ImplicitDataset(train_interaction_matrix, n_items, neg_ratio=4)
loader_train = DataLoader(dataset_train, batch_size=256, shuffle=True, collate_fn=collate_fn)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = NeuMF(num_users=n_users, num_items=len(items), gmf_dim=32, mlp_emb_dim=32, mlp_hidden_layers=[64, 32, 16, 8]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train_neumf(model, loader_train, optimizer, device, epochs=10)

pos, neg inter completed


Epoch 1/10:   0%|          | 0/297 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_22924\3047353411.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neg_item_ids = torch.tensor(neg_item_ids, dtype=torch.long).to(device)
Epoch 1/10: 100%|██████████| 297/297 [00:02<00:00, 103.62it/s]


Epoch 1, Loss: 0.6902073549501824


Epoch 2/10: 100%|██████████| 297/297 [00:02<00:00, 118.83it/s]


Epoch 2, Loss: 0.2919973339446466


...

Epoch 8, Loss: 0.002546065498334716


Epoch 9/10: 100%|██████████| 297/297 [00:02<00:00, 113.94it/s]


Epoch 9, Loss: 0.002429673549431448


Epoch 10/10: 100%|██████████| 297/297 [00:02<00:00, 110.09it/s]

Epoch 10, Loss: 0.0021473970533676307


In [ ]:
np.random.seed(42)
neg_ratio = 4

def collate_fn(batch):
    users, pos_items, neg_lists = zip(*batch)
    users = torch.LongTensor(users)
    pos_items = torch.LongTensor(pos_items)
    neg_items = torch.LongTensor([n for neg in neg_lists for n in neg])
    labels = torch.cat([torch.ones(len(users)), torch.zeros(len(neg_items))])
    items = torch.cat([pos_items, neg_items])
    users = users.repeat(1 + neg_ratio)
    return users, items, labels

#numbers of users
n_users = len(users)
n_countries = len(users['country'].unique())
n_genders = len(users['gender'].unique())

interaction_pairs_train = list(zip(train_inters['user_id'].to_list(), train_inters['item_id'].to_list())) 
interaction_pairs_test = list(zip(test_inters['user_id'].to_list(), test_inters['item_id'].to_list())) 

dataset_train = ImplicitDataset(interaction_pairs_train, len(items), neg_ratio=4)
loader_train = DataLoader(dataset_train, batch_size=256, shuffle=True, collate_fn=collate_fn)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = NeuMF(num_users=n_users, num_items=len(items), gmf_dim=32, mlp_emb_dim=32, mlp_hidden_layers=[64, 32, 16, 8]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_neumf(model, loader_train, optimizer, device, epochs=10)

In [11]:

def get_prediction(user_id, n_items, model, interactions, device='cuda', topN=10):
    """
    Generate top-N item recommendations for a given raw user_id.

    Args:
        user_id: raw user identifier (same type as in interactions_df['user_id']).
        model: trained NeuMF model.
        user_enc: LabelEncoder for user IDs.
        item_enc: LabelEncoder for item IDs.
        interactions_df: full interactions DataFrame with 'user_idx' and 'item_idx'.
        device: 'cuda' or 'cpu'.
        topN: number of recommendations to return.

    Returns:
        List of raw item IDs recommended.
    """

    item_indices = torch.arange(n_items, dtype=torch.long).to(device)
    user_tensor = torch.LongTensor([user_id] * n_items).to(device)
    model.eval()
    with torch.no_grad():
        scores = model(user_tensor, item_indices).cpu().numpy()
    interacted = set(interactions[interactions['user_id'] == user_id]['item_id'].values)
    scores[list(interacted)] = -np.inf
    top_idxs = np.argpartition(-scores, topN)[:topN]
    top_idxs = top_idxs[np.argsort(-scores[top_idxs])]
    return top_idxs


In [51]:

def get_negative_samples(user_id, n_items, pos_items_set, num_neg=1):
    """
    Sample num_neg negative items for a user (items not in pos_items_set).
    """
    negs = []
    while len(negs) < num_neg:
        neg_item = random.randint(0, n_items - 1)
        if neg_item not in pos_items_set and neg_item not in negs:
            negs.append(neg_item)
    return negs

class ImplicitDataset(Dataset):
    def __init__(self, inter_matr: np.ndarray, n_items: int, neg_ratio: int = 4):
        self.positive_pairs = []
        self.user_pos_items = {}
        for u in range(len(inter_matr)):
            pos_items = set(np.where(inter_matr[u] > 0)[0])
            self.user_pos_items[u] = pos_items
            self.positive_pairs.extend([(u, i) for i in pos_items])
        self.n_items = n_items
        self.neg_ratio = neg_ratio


    def __len__(self):
        return len(self.positive_pairs)

    def __getitem__(self, idx):
        u, i_pos = self.positive_pairs[idx]
        pos_items_set = self.user_pos_items[u]
        neg_items = get_negative_samples(u, self.n_items, pos_items_set, self.neg_ratio)
        return u, i_pos, neg_items
    
def collate_fn(batch):
    users, pos_items, neg_lists = zip(*batch)
    users = torch.LongTensor(users)
    pos_items = torch.LongTensor(pos_items)
    neg_items = torch.LongTensor([n for neg in neg_lists for n in neg])
    labels = torch.cat([torch.ones(len(users)), torch.zeros(len(neg_items))])
    items = torch.cat([pos_items, neg_items])
    users = users.repeat(1 + len(neg_lists[0]))
    return users, items, labels

np.random.seed(42)

#numbers of users
n_users = len(users)
n_items = len(items)
neg_ratios = [1,2,4,8]
batch_sizes = [64, 256, 1024]
emb_dims = [32, 64, 128]
results = []
#TODO change Loss function
for neg_ratio in neg_ratios:
    for batch_size in batch_sizes:
        for emb_dim in emb_dims:
            dataset_train = ImplicitDataset(train_interaction_matrix, n_items, neg_ratio=neg_ratio)
            loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
            device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
            model = NeuMF(num_users=n_users, num_items=len(items), gmf_dim=emb_dim, mlp_emb_dim=emb_dim, mlp_hidden_layers=[128, 64, 32]).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
            train_neumf(model, loader_train, optimizer, device, epochs=10)

            train_recs_list = []
            for i in range(len(users)):
                seen_item_ids = np.where(train_interaction_matrix[i] > 0)[0]
                train_rec_i = get_prediction(i,len(items), model, train_inters, device, topN=10)
                train_recs_list.append(",".join(map(str, train_rec_i)))
                
            user_id_list = np.array([i for i in range(len(users))])

            pop_train_df = pd.DataFrame({"user_id": user_id_list, "recs": train_recs_list})

            ndcg = get_ndcg_score_sk(pop_train_df, test_interaction_matrix, 10)
            results.append({"neg_ratio":neg_ratio, "batch_size":batch_size, "emb_dims":emb_dim, "ndcg_score":ndcg})

Epoch 1/10:   0%|          | 0/1186 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_22924\3047353411.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neg_item_ids = torch.tensor(neg_item_ids, dtype=torch.long).to(device)
Epoch 1/10: 100%|██████████| 1186/1186 [00:08<00:00, 134.54it/s]


Epoch 1, Loss: 0.1152799284066196


Epoch 2/10: 100%|██████████| 1186/1186 [00:07<00:00, 160.91it/s]


Epoch 2, Loss: 0.0032143002215423034


Epoch 3/10: 100%|██████████| 1186/1186 [00:07<00:00, 161.36it/s]


...

Epoch 8, Loss: 0.004698441323513786


Epoch 9/10: 100%|██████████| 75/75 [00:01<00:00, 44.77it/s]


Epoch 9, Loss: 0.0035734260051200786


Epoch 10/10: 100%|██████████| 75/75 [00:01<00:00, 45.97it/s]


Epoch 10, Loss: 0.003086864047994216


In [20]:
train_recs_list = []

for i in range(len(users)):
    seen_item_ids = np.where(train_interaction_matrix[i] > 0)[0]
    train_rec_i = get_prediction(i,len(items), model, train_inters, device, topN=10)
    train_recs_list.append(",".join(map(str, train_rec_i)))
    
user_id_list = np.array([i for i in range(len(users))])

pop_train_df = pd.DataFrame({"user_id": user_id_list, "recs": train_recs_list})

print(get_ndcg_score_sk(pop_train_df, test_interaction_matrix, 10))

0.0020592271831380293


In [53]:
for res in results:
    print(res)

{'neg_ratio': 1, 'batch_size': 64, 'emb_dims': 32, 'ndcg_score': 0.002270347047402082}
{'neg_ratio': 1, 'batch_size': 64, 'emb_dims': 64, 'ndcg_score': 0.003063282236650288}
{'neg_ratio': 1, 'batch_size': 64, 'emb_dims': 128, 'ndcg_score': 0.0030444082165369296}
{'neg_ratio': 1, 'batch_size': 256, 'emb_dims': 32, 'ndcg_score': 0.0019258151739141873}
{'neg_ratio': 1, 'batch_size': 256, 'emb_dims': 64, 'ndcg_score': 0.002866672382796591}
{'neg_ratio': 1, 'batch_size': 256, 'emb_dims': 128, 'ndcg_score': 0.0023235644548865028}
{'neg_ratio': 1, 'batch_size': 1024, 'emb_dims': 32, 'ndcg_score': 0.0021971403360381446}
{'neg_ratio': 1, 'batch_size': 1024, 'emb_dims': 64, 'ndcg_score': 0.002334429633765927}
{'neg_ratio': 1, 'batch_size': 1024, 'emb_dims': 128, 'ndcg_score': 0.0022758989427401092}
{'neg_ratio': 2, 'batch_size': 64, 'emb_dims': 32, 'ndcg_score': 0.0029441266236543023}
{'neg_ratio': 2, 'batch_size': 64, 'emb_dims': 64, 'ndcg_score': 0.0026347040965656607}
{'neg_ratio': 2, 'batch_